In [1]:
import pandas as pd
import numpy as np
from scipy.stats import poisson
from tqdm import tqdm

In [2]:
df_geral = pd.read_excel('./../datasets/info_futebol.xlsx', sheet_name='info_geral', index_col='Time')
df_partidas = pd.read_excel('./../datasets/info_futebol.xlsx', sheet_name='partidas')

In [3]:
# Criando coluna de força de times
a, b = df_geral['Pontuacao'].min(), df_geral['Pontuacao'].max()
fa, fb = 0.15, 1 # novos limites
b1 = (fb - fa)/(b-a)
b0 = fb - b*b1
df_geral['forca'] = round(b0 + b1*df_geral['Pontuacao'], 3)

In [4]:
def lambda_(df, team1, team2):
    
    #forca1 = df.loc[team1]['forca']
    #forca2 = df.loc[team2]['forca']
    forca1 = df.loc[team1]['forca'] 
    forca2 = df.loc[team2]['forca']
    
    if forca1 >= 0.8:
        forca1 = forca1
    else:
        forca1 += 0.15*forca1
    #forca1 += 0.20
    m = 2.25
    lambda1 = m*forca1/(forca2 + forca1)
    lambda2 = m - lambda1

    return [lambda1, lambda2]

In [5]:
lambda_(df_geral, 'Palmeiras', 'Ituano')

[1.7942583732057416, 0.45574162679425845]

In [6]:
def resultado_vde(gols1, gols2):

    if gols1 > gols2:
       	resultado = 'V'
    elif gols1 < gols2:
       	resultado = 'D'
    else:
       	resultado = 'E'

    return resultado

In [7]:
resultado_vde(5, 1)

'V'

In [8]:
def pontos_time(gols1, gols2):
    
    resultado = resultado_vde(gols1, gols2)
    if resultado == 'V':
       	pontos1, pontos2 = 3, 0
    elif resultado == 'D':
       	pontos1, pontos2 = 0, 3
    else:
       	pontos1, pontos2 = 1, 1

    return [pontos1, pontos2, resultado]

In [9]:
pontos_time(5, 1)

[3, 0, 'V']

In [10]:
def dist_poisson(media):

    probs = [poisson.pmf(i, media) for i in range(0, 6, 1)] # calcula a probabilidade de sair i gols quando a média = media
    probs.append(1-sum(probs)) # probabilidade de sair mais que 5 gols
    return pd.Series(probs, index = ['0', '1', '2', '3', '4', '5', '6+'])

In [11]:
dist_poisson(4)

0     0.018316
1     0.073263
2     0.146525
3     0.195367
4     0.195367
5     0.156293
6+    0.214870
dtype: float64

In [12]:
def probabilidades_partidas(df, team1, team2):

    lambda1, lambda2 = lambda_(df, team1, team2)
    dist1, dist2 = dist_poisson(lambda1), dist_poisson(lambda2)
    matriz_resultados = np.outer(dist1, dist2) # outer produto dos valores de dist1 e dist2

    # empates
    empates = np.trace(matriz_resultados)

    # soma dos triangulos inferiores
    vitoria_team1 = np.tril(matriz_resultados).sum() - empates

    # soma dos triangulos superiores
    vitoria_team2 = np.triu(matriz_resultados).sum() - empates

    vde = np.around([vitoria_team1, empates, vitoria_team2], 3)
    probabilidades = [f'{100*i:.2f}%' for i in vde]

    return probabilidades, 100*matriz_resultados

In [13]:
probabilidades_partidas(df_geral, 'São Bento', 'Palmeiras')

(['10.80%', '22.50%', '66.70%'],
 array([[1.05399225e+01, 1.81757620e+01, 1.56717626e+01, 9.00848590e+00,
         3.88371207e+00, 1.33946766e+00, 5.04978383e-01],
        [5.53906348e+00, 9.55193933e+00, 8.23600823e+00, 4.73424501e+00,
         2.04101385e+00, 7.03932731e-01, 2.65382154e-01],
        [1.45547675e+00, 2.50992351e+00, 2.16414175e+00, 1.24399794e+00,
         5.36308750e-01, 1.84969486e-01, 6.97333687e-02],
        [2.54966396e-01, 4.39681465e-01, 3.79108372e-01, 2.17920122e-01,
         9.39490850e-02, 3.24024436e-02, 1.22156989e-02],
        [3.34982316e-02, 5.77666382e-02, 4.98083679e-02, 2.86309836e-02,
         1.23433058e-02, 4.25712790e-03, 1.60493428e-03],
        [3.52087660e-03, 6.07164004e-03, 5.23517536e-03, 3.00929796e-03,
         1.29735973e-03, 4.47451142e-04, 1.68688772e-04],
        [3.33155934e-04, 5.74516843e-04, 4.95368039e-04, 2.84748825e-04,
         1.22760080e-04, 4.23391729e-05, 1.59618390e-05]]))

In [14]:
def simula_jogo(df, team1, team2):

    lambda1, lambda2 = lambda_(df, team1, team2)
    gols1 = int(np.random.poisson(lam=lambda1, size=1))
    gols2 = int(np.random.poisson(lam=lambda2, size=1))
    saldo1 = gols1 - gols2
    saldo2 = gols2 - gols1
    pts1, pts2, resultado = pontos_time(gols1, gols2)
    #placar = f'{gols1} X {gols2}'
    
    return [gols1, gols2, saldo1, saldo2, pts1, pts2, team1, team2, resultado]
    #return [gols1, gols2, saldo1, saldo2, pts1, pts2, resultado, placar]

In [15]:
simula_jogo(df_geral, 'São Paulo', 'Palmeiras')

[3, 1, 2, -2, 3, 0, 'São Paulo', 'Palmeiras', 'V']

# Simulação da primeira fase

In [ ]:
def criacao_tabela_por_vez():
    return

In [18]:
df_grupos = pd.DataFrame(np.zeros((16, 8)), index=['Santos', 'Botafogo-SP', 'Inter de Limeira', 'Bragantino', 'São Paulo', 
                                            'Água Santa', 'Guarani', 'Mirassol', 'Corinthians', 'Ferroviária', 'Ituano', 
                                            'São Bento', 'Palmeiras', 'Portuguesa', 'Santo André', 'São Bernardo'], columns=['pts', 'V', 'E', 'D', 'Jogos', 'gfeitos', 'gsofridos', 'SG']).astype(int)

In [19]:
df_grupos['grupo'] = np.nan

In [20]:
df_grupos

,pts,V,E,D,Jogos,gfeitos,gsofridos,SG,grupo
Santos,0,0,0,0,0,0,0,0,NaN
Botafogo-SP,0,0,0,0,0,0,0,0,NaN
Inter de Limeira,0,0,0,0,0,0,0,0,NaN
Bragantino,0,0,0,0,0,0,0,0,NaN
São Paulo,0,0,0,0,0,0,0,0,NaN
Água Santa,0,0,0,0,0,0,0,0,NaN
Guarani,0,0,0,0,0,0,0,0,NaN
Mirassol,0,0,0,0,0,0,0,0,NaN
Corinthians,0,0,0,0,0,0,0,0,NaN
Ferroviária,0,0,0,0,0,0,0,0,NaN


In [22]:
df_grupos.iloc[0:4, 8] = 'A'
df_grupos.iloc[4:8, 8] = 'B'
df_grupos.iloc[8:12, 8] = 'C'
df_grupos.iloc[12:16, 8] = 'D'

In [25]:
def simula_grupo(df_grupos, df_geral):
    df_tmp = df = df_grupos.copy()
    for i in range(96):
        times = df_partidas.iloc[i].to_list()
        gols1, gols2, saldo1, saldo2, pts1, pts2, time1, time2, resultado = simula_jogo(df_geral, times[0].strip(), times[1].strip())
        
        if resultado == 'V':
            df_tmp.loc[time1, 'V'] += 1
            df_tmp.loc[time2, 'D'] += 1
        elif resultado == 'D':
            df_tmp.loc[time1, 'D'] += 1
            df_tmp.loc[time2, 'V'] += 1
        else:
            df_tmp.loc[time1, 'E'] += 1
            df_tmp.loc[time2, 'E'] += 1
            
        #time 1
        df_tmp.loc[time1,'gfeitos'] += gols1
        df_tmp.loc[time1,'gsofridos'] += gols2
        df_tmp.loc[time1,'SG'] += saldo1
        df_tmp.loc[time1,'pts'] += pts1
        df_tmp.loc[time1, 'Jogos'] += 1
        #time 2
        df_tmp.loc[time2,'gfeitos'] += gols2
        df_tmp.loc[time2,'gsofridos'] += gols1
        df_tmp.loc[time2,'SG'] += saldo2
        df_tmp.loc[time2,'pts'] += pts2
        df_tmp.loc[time2, 'Jogos'] += 1
        
        
    return df_tmp

In [41]:
df_results = simula_grupo(df_grupos, df_geral)

In [42]:
df_results

,pts,V,E,D,Jogos,gfeitos,gsofridos,SG,grupo
Santos,26,7,5,0,12,19,6,13,A
Botafogo-SP,15,3,6,3,12,12,13,-1,A
Inter de Limeira,17,5,2,5,12,9,15,-6,A
Bragantino,22,6,4,2,12,13,6,7,A
São Paulo,19,5,4,3,12,13,7,6,B
Água Santa,10,2,4,6,12,8,16,-8,B
Guarani,18,5,3,4,12,15,5,10,B
Mirassol,15,4,3,5,12,7,10,-3,B
Corinthians,22,6,4,2,12,15,9,6,C
Ferroviária,9,2,3,7,12,9,16,-7,C


# Pegando os classificados

In [27]:
# pts, número de vitórias, saldo de gols, gols marcados,

In [28]:
df_results_groups = df_results.groupby('grupo')

In [29]:
dfA = df_results_groups.get_group('A').sort_values(by=['pts', 'V', 'SG', 'gfeitos'], ascending=False)
dfB = df_results_groups.get_group('B').sort_values(by=['pts', 'V', 'SG', 'gfeitos'], ascending=False)
dfC = df_results_groups.get_group('C').sort_values(by=['pts', 'V', 'SG', 'gfeitos'], ascending=False)
dfD = df_results_groups.get_group('D').sort_values(by=['pts', 'V', 'SG', 'gfeitos'], ascending=False)

In [ ]:
def classificados(df, df_final):
    primeiro, segundo, terceiro, quarto = df.index
    df_final.loc[primeiro, '1st'] += 1
    df_final.loc[segundo, '2nd'] += 1
    df_final.loc[terceiro, '3rd'] += 1
    df_final.loc[quarto, '4th'] += 1
    
    return df_final

# Simulando fase final

#### Quartas de Final:
- grupo E:
pos1 x pos2 grupo A

- grupo F:
pos1 x pos2 grupo B

- grupo G:
pos1 x pos2 grupo C

- grupo H:
pos1 x pos2 grupo D


#### Semi - Final:

- grupo I:
1 melhor campanha x 4 melhor campanha

- grupo J:
2 melhor camapanha x 3 melhor campanha

#### FINAL:

- grupo K:

pos1 grupo I x pos1 grupo J 


# Resultados

In [32]:
df_final = pd.DataFrame(np.zeros((16, 8)), index=['Santos', 'Botafogo-SP', 'Inter de Limeira', 'Bragantino', 'São Paulo', 
                                            'Água Santa', 'Guarani', 'Mirassol', 'Corinthians', 'Ferroviária', 'Ituano', 
                                            'São Bento', 'Palmeiras', 'Portuguesa', 'Santo André', 'São Bernardo'], columns=['1st', '2nd', '3rd', '4th', 'Quartas', 'Semis', 'Final', 'Campeão']).astype(int)

In [33]:
df_final

,1st,2nd,3rd,4th,Quartas,Semis,Final,Campeão
Santos,0,0,0,0,0,0,0,0
Botafogo-SP,0,0,0,0,0,0,0,0
Inter de Limeira,0,0,0,0,0,0,0,0
Bragantino,0,0,0,0,0,0,0,0
São Paulo,0,0,0,0,0,0,0,0
Água Santa,0,0,0,0,0,0,0,0
Guarani,0,0,0,0,0,0,0,0
Mirassol,0,0,0,0,0,0,0,0
Corinthians,0,0,0,0,0,0,0,0
Ferroviária,0,0,0,0,0,0,0,0
